In [ ]:
import datetime as dt
from gas_storage.core.gas_storage import GasStorage
import json as json
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px

In [ ]:
data_path = Path("../data/")
gas_fwd = pd.read_csv(data_path / "gas_fwd.csv", parse_dates=True, index_col="date")
gas_fwd.index.name = None
gas_fwd = gas_fwd.reindex(
    pd.date_range(
        gas_fwd.index.min(), gas_fwd.index.max() + pd.offsets.MonthEnd(1), freq="D"
    ),
    method="ffill",
)
gas_fwd = gas_fwd["price"]

with open(data_path / "storages.json", "r") as file:
    storages = json.load(file)
storage_json = storages[0]

In [ ]:
date_start = dt.datetime(2024, 4, 1)
date_end = dt.datetime(2025, 3, 31)

storage = GasStorage(date_start=date_start, date_end=date_end)

In [ ]:
price = gas_fwd.loc[date_start:date_end]
storage.attr["price"] = pd.Series(price.values[::-1], index=price.index)

storage.z0 = 0
storage.injection_season = storage_json["InjectionSeason"]
storage.bsd_state_to_date = {int(key): val for key, val in storage_json["StatesToDate"].items()}
dates_to_empty_storage = []
for date in storage_json["DatesToEmptyStorage"]:
    dates_to_empty_storage.append(dt.datetime.strptime(date, "%Y-%m-%d").date())
dates_to_empty_storage.append(date_end)
storage.empty_on_dates = dates_to_empty_storage

In [ ]:
for period in storage_json["TimePeriods"]:
    period_start_date = dt.datetime.strptime(period["StartDate"], "%Y-%m-%d")
    period_end_date = dt.datetime.strptime(period["EndDate"], "%Y-%m-%d")
    if period_end_date >= date_start:
        storage.load_attribute("wgv", period["WGV"], period_start_date, period_end_date)
        storage.load_attribute(
            "wr", period["WithdrawalRate"], period_start_date, period_end_date
        )
        storage.load_attribute(
            "ir", period["InjectionRate"], period_start_date, period_end_date
        )

storage.load_attribute(
    "inj_curve",
    np.array(storage_json["InjectionCurve"]) / 100,
    date_start,
    date_end,
)
storage.load_attribute(
    "wit_curve",
    np.array(storage_json["WithdrawalCurve"]) / 100,
    date_start,
    date_end,
)

In [ ]:
storage.create_model()

In [ ]:
storage.solve_model(solver_name="scip", stream_solver=True)

In [ ]:
sum(storage.res_injection.values())

In [ ]:
sum(storage.res_withdrawal.values())

In [ ]:
-sum(np.array(list(storage.res_operations.values())) * storage.attr["price"].to_numpy())

In [ ]:
storage.graph()

In [ ]:
px.line(storage.attr["price"])